# 탐색적 데이터 분석 (EDA) - 축구 선수 잠재력 예측

데이콘 경진대회 데이터셋: https://dacon.io/competitions/official/236031/data

- **타겟 변수**: `Prospect` (0 또는 1)
- **목표**: 축구 선수의 잠재력 예측에 도움이 되는 컬럼 분석

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 1. 데이터 로드

In [ ]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

print(f'Train shape: {train.shape}')
print(f'Test shape: {test.shape}')

In [ ]:
train.head()

In [ ]:
train.info()

## 2. 타겟 변수 분포 확인

In [ ]:
print('Prospect 분포:')
print(train['Prospect'].value_counts())
print(f'\nProspect 비율:\n{train["Prospect"].value_counts(normalize=True)}')

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
train['Prospect'].value_counts().plot(kind='bar', ax=ax, color=['steelblue', 'coral'])
ax.set_title('Prospect 분포')
ax.set_xlabel('Prospect')
ax.set_ylabel('Count')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

## 3. 결측치 확인

In [ ]:
print('Train 결측치:')
missing_train = train.isnull().sum()
print(missing_train[missing_train > 0] if missing_train.sum() > 0 else '결측치 없음')

print('\nTest 결측치:')
missing_test = test.isnull().sum()
print(missing_test[missing_test > 0] if missing_test.sum() > 0 else '결측치 없음')

## 4. 수치형 / 범주형 변수 구분

In [ ]:
# ID와 타겟 제외
feature_cols = [col for col in train.columns if col not in ['ID', 'Prospect']]

# 범주형 변수
cat_cols = train[feature_cols].select_dtypes(include=['object']).columns.tolist()
print(f'범주형 변수 ({len(cat_cols)}개): {cat_cols}')

# 수치형 변수
num_cols = train[feature_cols].select_dtypes(include=['int64', 'float64']).columns.tolist()
print(f'\n수치형 변수 ({len(num_cols)}개): {num_cols[:10]}...')

## 5. 범주형 변수 분석

In [ ]:
for col in cat_cols:
    print(f'\n{col} 고유값 ({train[col].nunique()}개):')
    print(train[col].value_counts())

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for idx, col in enumerate(cat_cols):
    ax = axes[idx // 2, idx % 2]
    prospect_rate = train.groupby(col)['Prospect'].mean().sort_values(ascending=False)
    prospect_rate.plot(kind='bar', ax=ax, color='steelblue')
    ax.set_title(f'{col}별 Prospect 비율')
    ax.set_ylabel('Prospect 비율')
    ax.axhline(y=train['Prospect'].mean(), color='red', linestyle='--', label='전체 평균')
    ax.legend()
    plt.sca(ax)
    plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.show()

## 6. 수치형 변수 기초 통계

In [ ]:
train[num_cols].describe()

## 7. 상관관계 분석

In [ ]:
# 타겟과의 상관관계
correlation_with_target = train[num_cols + ['Prospect']].corr()['Prospect'].drop('Prospect').sort_values(key=abs, ascending=False)

print('Prospect와의 상관관계 (상위 30개):')
print(correlation_with_target.head(30))

In [ ]:
# 상관관계 시각화 - 상위 30개 변수
top_corr_cols = correlation_with_target.head(30).index.tolist()

fig, ax = plt.subplots(figsize=(12, 10))
correlation_with_target.head(30).plot(kind='barh', ax=ax, color='steelblue')
ax.set_title('Prospect와의 상관관계 (상위 30개)')
ax.set_xlabel('상관계수')
ax.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
plt.tight_layout()
plt.show()

In [ ]:
# 상위 상관관계 변수들 히트맵
top_15_cols = correlation_with_target.head(15).index.tolist() + ['Prospect']

fig, ax = plt.subplots(figsize=(14, 12))
corr_matrix = train[top_15_cols].corr()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r', 
            center=0, ax=ax, square=True, linewidths=0.5)
ax.set_title('상위 15개 변수와 Prospect 상관관계 히트맵')
plt.tight_layout()
plt.show()

## 8. 주요 변수 그룹별 분석

In [ ]:
# 주요 변수 그룹 정의
basic_cols = ['Age', 'Height', 'Weight']
total_cols = ['PaceTotal', 'ShootingTotal', 'PassingTotal', 'DribblingTotal', 'DefendingTotal', 'PhysicalityTotal']
rating_cols = [col for col in num_cols if 'Rating' in col]
gk_cols = [col for col in num_cols if col.startswith('GK')]

print(f'기본 정보 컬럼: {basic_cols}')
print(f'종합 스탯 컬럼: {total_cols}')
print(f'레이팅 컬럼 ({len(rating_cols)}개): {rating_cols}')
print(f'골키퍼 스탯 컬럼: {gk_cols}')

In [ ]:
# 기본 정보와 타겟 관계
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, col in enumerate(basic_cols):
    ax = axes[idx]
    train.boxplot(column=col, by='Prospect', ax=ax)
    ax.set_title(f'{col} by Prospect')
    ax.set_xlabel('Prospect')

plt.suptitle('')
plt.tight_layout()
plt.show()

In [ ]:
# 종합 스탯과 타겟 관계
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for idx, col in enumerate(total_cols):
    ax = axes[idx // 3, idx % 3]
    train.boxplot(column=col, by='Prospect', ax=ax)
    ax.set_title(f'{col} by Prospect')
    ax.set_xlabel('Prospect')

plt.suptitle('')
plt.tight_layout()
plt.show()

## 9. 통계적 검정 (t-test)

In [ ]:
# Prospect 0 vs 1 그룹 분리
prospect_0 = train[train['Prospect'] == 0]
prospect_1 = train[train['Prospect'] == 1]

# 각 수치형 변수에 대해 t-test 수행
ttest_results = []
for col in num_cols:
    t_stat, p_value = stats.ttest_ind(prospect_0[col], prospect_1[col])
    mean_diff = prospect_1[col].mean() - prospect_0[col].mean()
    ttest_results.append({
        'Variable': col,
        'Mean_Prospect_0': prospect_0[col].mean(),
        'Mean_Prospect_1': prospect_1[col].mean(),
        'Mean_Diff': mean_diff,
        't_statistic': t_stat,
        'p_value': p_value
    })

ttest_df = pd.DataFrame(ttest_results)
ttest_df = ttest_df.sort_values('p_value')

print('통계적으로 유의미한 변수 (p < 0.05):')
significant = ttest_df[ttest_df['p_value'] < 0.05]
print(f'총 {len(significant)}개')
significant.head(30)

## 10. 포지션별 Prospect 분석

In [ ]:
position_analysis = train.groupby('Position').agg({
    'Prospect': ['count', 'sum', 'mean']
}).round(3)
position_analysis.columns = ['Count', 'Prospect_Count', 'Prospect_Rate']
position_analysis = position_analysis.sort_values('Prospect_Rate', ascending=False)
print('포지션별 Prospect 분석:')
position_analysis

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))
position_analysis['Prospect_Rate'].plot(kind='bar', ax=ax, color='steelblue')
ax.axhline(y=train['Prospect'].mean(), color='red', linestyle='--', label='전체 평균')
ax.set_title('포지션별 Prospect 비율')
ax.set_ylabel('Prospect 비율')
ax.set_xlabel('Position')
ax.legend()
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 11. 나이별 Prospect 분석

In [ ]:
age_analysis = train.groupby('Age').agg({
    'Prospect': ['count', 'sum', 'mean']
}).round(3)
age_analysis.columns = ['Count', 'Prospect_Count', 'Prospect_Rate']
print('나이별 Prospect 분석:')
age_analysis

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 나이별 선수 수
ax1 = axes[0]
age_analysis['Count'].plot(kind='bar', ax=ax1, color='steelblue')
ax1.set_title('나이별 선수 수')
ax1.set_ylabel('Count')
ax1.set_xlabel('Age')

# 나이별 Prospect 비율
ax2 = axes[1]
age_analysis['Prospect_Rate'].plot(kind='bar', ax=ax2, color='coral')
ax2.axhline(y=train['Prospect'].mean(), color='red', linestyle='--', label='전체 평균')
ax2.set_title('나이별 Prospect 비율')
ax2.set_ylabel('Prospect 비율')
ax2.set_xlabel('Age')
ax2.legend()

plt.tight_layout()
plt.show()

## 12. 상위 상관관계 변수 분포 비교

In [ ]:
# 상위 9개 상관관계 변수 분포
top_9_cols = correlation_with_target.head(9).index.tolist()

fig, axes = plt.subplots(3, 3, figsize=(15, 12))

for idx, col in enumerate(top_9_cols):
    ax = axes[idx // 3, idx % 3]
    train[train['Prospect'] == 0][col].hist(ax=ax, bins=30, alpha=0.5, label='Prospect=0', color='blue')
    train[train['Prospect'] == 1][col].hist(ax=ax, bins=30, alpha=0.5, label='Prospect=1', color='red')
    ax.set_title(f'{col} 분포')
    ax.legend()

plt.tight_layout()
plt.show()

## 13. Feature Importance 요약

In [ ]:
# 상관관계 기반 중요 변수 요약
print('='*60)
print('Prospect 예측에 중요한 변수 요약')
print('='*60)

print('\n[상관관계 기준 상위 10개 변수]')
for i, (col, corr) in enumerate(correlation_with_target.head(10).items(), 1):
    print(f'{i:2d}. {col:25s}: {corr:+.4f}')

print('\n[t-test p-value 기준 상위 10개 변수]')
for i, row in ttest_df.head(10).iterrows():
    print(f"{ttest_df.index.get_loc(i)+1:2d}. {row['Variable']:25s}: p={row['p_value']:.2e}, diff={row['Mean_Diff']:+.3f}")

## 14. 다중공선성 확인

In [ ]:
# 상위 상관관계 변수들 간의 상관관계 확인
top_20_cols = correlation_with_target.head(20).index.tolist()

fig, ax = plt.subplots(figsize=(16, 14))
corr_matrix = train[top_20_cols].corr()
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='RdBu_r', 
            center=0, ax=ax, square=True, linewidths=0.5, annot_kws={'size': 8})
ax.set_title('상위 20개 변수 간 상관관계')
plt.tight_layout()
plt.show()

In [ ]:
# 높은 상관관계를 가진 변수쌍 확인
high_corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > 0.8:
            high_corr_pairs.append({
                'Variable_1': corr_matrix.columns[i],
                'Variable_2': corr_matrix.columns[j],
                'Correlation': corr_matrix.iloc[i, j]
            })

high_corr_df = pd.DataFrame(high_corr_pairs).sort_values('Correlation', key=abs, ascending=False)
print('높은 상관관계 (|r| > 0.8)를 가진 변수쌍:')
high_corr_df

## 15. 결론 및 권장 사항

In [ ]:
print('='*70)
print('EDA 결론 및 모델링 권장 사항')
print('='*70)

print(f'''
1. 데이터 개요
   - Train 샘플 수: {len(train)}
   - Test 샘플 수: {len(test)}
   - 전체 피처 수: {len(feature_cols)}
   - 수치형 피처: {len(num_cols)}개
   - 범주형 피처: {len(cat_cols)}개
   - 타겟 클래스 비율: Prospect=0 ({(1-train['Prospect'].mean())*100:.1f}%), Prospect=1 ({train['Prospect'].mean()*100:.1f}%)

2. 중요 변수 (상관관계 기준)
   - 상위 5개: {', '.join(correlation_with_target.head(5).index.tolist())}

3. 범주형 변수 영향
   - Position: 포지션에 따라 Prospect 비율 차이 존재
   - AttackingWorkRate, DefensiveWorkRate: Work Rate에 따른 차이 확인 필요
   - PreferredFoot: 선호 발에 따른 차이 확인

4. 권장 사항
   - 다중공선성이 높은 변수들 주의 (Rating 관련 변수들)
   - 범주형 변수 인코딩 필요 (One-Hot 또는 Label Encoding)
   - 클래스 불균형 처리 고려 (SMOTE, 클래스 가중치 등)
''')

## 16. 특성공학 (Feature Engineering)

유망주 예측에 도움이 될 수 있는 파생 변수들을 생성합니다.

In [ ]:
def feature_engineering(df):
    """특성공학 함수 - train과 test에 동일하게 적용"""
    df = df.copy()
    
    # ============================================
    # 1. 신체 정보 파생 변수
    # ============================================
    # BMI (체질량지수)
    df['BMI'] = df['Weight'] / (df['Height'] / 100) ** 2
    
    # 키 대비 체중 비율
    df['Height_Weight_Ratio'] = df['Height'] / df['Weight']
    
    # ============================================
    # 2. 포지션 그룹화
    # ============================================
    position_map = {
        'ST': 'Forward', 'CF': 'Forward', 'LW': 'Forward', 'RW': 'Forward',
        'CAM': 'Midfielder', 'CM': 'Midfielder', 'CDM': 'Midfielder', 
        'LM': 'Midfielder', 'RM': 'Midfielder',
        'LB': 'Defender', 'RB': 'Defender', 'CB': 'Defender', 
        'LWB': 'Defender', 'RWB': 'Defender',
        'GK': 'Goalkeeper'
    }
    df['Position_Group'] = df['Position'].map(position_map)
    
    # 골키퍼 여부
    df['Is_GK'] = (df['Position'] == 'GK').astype(int)
    
    # ============================================
    # 3. 종합 스탯 집계
    # ============================================
    total_cols = ['PaceTotal', 'ShootingTotal', 'PassingTotal', 
                  'DribblingTotal', 'DefendingTotal', 'PhysicalityTotal']
    
    # 전체 스탯 합계
    df['Total_Stats_Sum'] = df[total_cols].sum(axis=1)
    
    # 전체 스탯 평균
    df['Total_Stats_Mean'] = df[total_cols].mean(axis=1)
    
    # 공격 스탯 합계
    df['Offense_Stats'] = df['PaceTotal'] + df['ShootingTotal'] + df['DribblingTotal']
    
    # 수비 스탯 합계
    df['Defense_Stats'] = df['DefendingTotal'] + df['PhysicalityTotal']
    
    # 공격 vs 수비 비율
    df['Attack_Defense_Ratio'] = df['Offense_Stats'] / (df['Defense_Stats'] + 0.01)
    
    # ============================================
    # 4. 나이 대비 스탯 (잠재력 핵심 지표!)
    # ============================================
    df['Stats_Per_Age'] = df['Total_Stats_Sum'] / df['Age']
    df['Offense_Per_Age'] = df['Offense_Stats'] / df['Age']
    df['Defense_Per_Age'] = df['Defense_Stats'] / df['Age']
    
    # 나이 그룹
    df['Age_Group'] = pd.cut(df['Age'], bins=[0, 18, 19, 20, 21, 100], 
                             labels=['U18', '18', '19', '20', '21+'])
    
    # ============================================
    # 5. 포지션별 레이팅 관련
    # ============================================
    # 포지션-레이팅 매핑
    position_rating_map = {
        'ST': 'STRating', 'CF': 'CFRating', 'LW': 'LWRating', 'RW': 'RWRating',
        'LF': 'LFRating', 'RF': 'RFRating',
        'CAM': 'CAMRating', 'CM': 'CMRating', 'CDM': 'CDMRating',
        'LM': 'LMRating', 'RM': 'RMRating',
        'LB': 'LBRating', 'RB': 'RBRating', 'CB': 'CBRating',
        'LWB': 'LWBRating', 'RWB': 'RWBRating',
        'GK': 'GKRating'
    }
    
    # 현재 포지션 레이팅
    df['Current_Position_Rating'] = df.apply(
        lambda row: row[position_rating_map.get(row['Position'], 'CMRating')], axis=1
    )
    
    # 필드 플레이어 레이팅 컬럼 (GK 제외)
    field_rating_cols = ['STRating', 'LWRating', 'LFRating', 'CFRating', 'RFRating', 
                         'RWRating', 'CAMRating', 'LMRating', 'CMRating', 'RMRating',
                         'LWBRating', 'CDMRating', 'RWBRating', 'LBRating', 'CBRating', 'RBRating']
    
    # 최고 필드 레이팅
    df['Max_Field_Rating'] = df[field_rating_cols].max(axis=1)
    
    # 최고 레이팅과 현재 포지션 레이팅 차이 (다재다능성)
    df['Rating_Gap'] = df['Max_Field_Rating'] - df['Current_Position_Rating']
    
    # 최적 포지션 여부
    df['Is_Best_Position'] = (df['Current_Position_Rating'] == df['Max_Field_Rating']).astype(int)
    
    # 레이팅 평균
    df['Avg_Field_Rating'] = df[field_rating_cols].mean(axis=1)
    
    # 레이팅 표준편차 (다재다능 vs 전문가)
    df['Rating_Std'] = df[field_rating_cols].std(axis=1)
    
    # ============================================
    # 6. 세부 스탯 조합
    # ============================================
    # 기술적 능력
    df['Technical_Skill'] = (df['BallControl'] + df['Dribbling'] + 
                             df['ShortPassing'] + df['LongPassing']) / 4
    
    # 정신적 능력
    df['Mental_Skill'] = (df['Reactions'] + df['Composure'] + 
                          df['Vision'] + df['Positioning']) / 4
    
    # 신체적 능력
    df['Physical_Skill'] = (df['Acceleration'] + df['SprintSpeed'] + 
                            df['Stamina'] + df['Strength']) / 4
    
    # 슈팅 능력
    df['Shooting_Skill'] = (df['Finishing'] + df['LongShots'] + 
                            df['ShotPower'] + df['Volleys']) / 4
    
    # 수비 능력
    df['Defending_Skill'] = (df['Marking'] + df['StandingTackle'] + 
                             df['SlidingTackle'] + df['Interceptions']) / 4
    
    # 종합 스킬 합계
    df['Total_Skill'] = (df['Technical_Skill'] + df['Mental_Skill'] + 
                         df['Physical_Skill'] + df['Shooting_Skill'] + df['Defending_Skill'])
    
    # ============================================
    # 7. 스탯 다양성/전문성
    # ============================================
    skill_cols = ['Technical_Skill', 'Mental_Skill', 'Physical_Skill', 
                  'Shooting_Skill', 'Defending_Skill']
    
    # 스킬 표준편차 (낮으면 올라운더, 높으면 전문가)
    df['Skill_Std'] = df[skill_cols].std(axis=1)
    
    # 최고 스킬과 최저 스킬 차이
    df['Skill_Range'] = df[skill_cols].max(axis=1) - df[skill_cols].min(axis=1)
    
    # ============================================
    # 8. Work Rate 관련
    # ============================================
    workrate_map = {'High': 3, 'Medium': 2, 'Low': 1}
    
    df['AttackingWorkRate_Encoded'] = df['AttackingWorkRate'].map(workrate_map)
    df['DefensiveWorkRate_Encoded'] = df['DefensiveWorkRate'].map(workrate_map)
    
    # Work Rate 총합
    df['WorkRate_Total'] = df['AttackingWorkRate_Encoded'] + df['DefensiveWorkRate_Encoded']
    
    # Work Rate 조합
    df['WorkRate_Combo'] = df['AttackingWorkRate'] + '_' + df['DefensiveWorkRate']
    
    # ============================================
    # 9. 골키퍼 스탯 관련
    # ============================================
    gk_cols = ['GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes']
    
    # GK 스탯 합계
    df['GK_Stats_Sum'] = df[gk_cols].sum(axis=1)
    
    # ============================================
    # 10. 포지션별 핵심 스탯
    # ============================================
    # 공격수 핵심 스탯
    df['Forward_Key_Stats'] = (df['Finishing'] + df['Positioning'] + 
                               df['Composure'] + df['ShotPower']) / 4
    
    # 미드필더 핵심 스탯
    df['Midfielder_Key_Stats'] = (df['Vision'] + df['ShortPassing'] + 
                                  df['BallControl'] + df['LongPassing']) / 4
    
    # 수비수 핵심 스탯
    df['Defender_Key_Stats'] = (df['Marking'] + df['StandingTackle'] + 
                                df['Interceptions'] + df['HeadingAccuracy']) / 4
    
    # 포지션에 맞는 핵심 스탯 점수
    def get_position_fit_score(row):
        pos_group = row['Position_Group']
        if pos_group == 'Forward':
            return row['Forward_Key_Stats']
        elif pos_group == 'Midfielder':
            return row['Midfielder_Key_Stats']
        elif pos_group == 'Defender':
            return row['Defender_Key_Stats']
        else:  # Goalkeeper
            return row['GK_Stats_Sum'] / 5
    
    df['Position_Fit_Score'] = df.apply(get_position_fit_score, axis=1)
    
    return df

In [ ]:
# 특성공학 적용
train_fe = feature_engineering(train)
test_fe = feature_engineering(test)

print(f'특성공학 전 Train 컬럼 수: {len(train.columns)}')
print(f'특성공학 후 Train 컬럼 수: {len(train_fe.columns)}')
print(f'\n새로 생성된 컬럼 ({len(train_fe.columns) - len(train.columns)}개):')
new_cols = [col for col in train_fe.columns if col not in train.columns]
print(new_cols)

### 16.1 새로운 변수들의 Prospect 상관관계

In [ ]:
# 새로 생성된 수치형 변수들의 상관관계
new_num_cols = [col for col in new_cols if train_fe[col].dtype in ['int64', 'float64', 'int32', 'float32']]

new_corr = train_fe[new_num_cols + ['Prospect']].corr()['Prospect'].drop('Prospect').sort_values(key=abs, ascending=False)

print('새로 생성된 변수들과 Prospect 상관관계:')
print(new_corr)

In [ ]:
# 새로운 변수들 상관관계 시각화
fig, ax = plt.subplots(figsize=(12, 10))
new_corr.plot(kind='barh', ax=ax, color='coral')
ax.set_title('새로 생성된 변수들과 Prospect 상관관계')
ax.set_xlabel('상관계수')
ax.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
plt.tight_layout()
plt.show()

### 16.2 전체 변수 상관관계 비교 (기존 vs 새로운)

In [ ]:
# 전체 수치형 변수 상관관계 (기존 + 새로운)
all_num_cols = train_fe.select_dtypes(include=['int64', 'float64', 'int32', 'float32']).columns.tolist()
all_num_cols = [col for col in all_num_cols if col not in ['ID', 'Prospect']]

all_corr = train_fe[all_num_cols + ['Prospect']].corr()['Prospect'].drop('Prospect').sort_values(key=abs, ascending=False)

# 상위 20개 변수 출력
print('전체 변수 중 Prospect와 상관관계 Top 20:')
print(all_corr.head(20))
print('\n기존 변수 vs 새로운 변수 (Top 20 중):')
for i, (col, corr) in enumerate(all_corr.head(20).items(), 1):
    is_new = '★ NEW' if col in new_cols else ''
    print(f'{i:2d}. {col:30s}: {corr:+.4f} {is_new}')

### 16.3 포지션 그룹별 Prospect 분석

In [ ]:
# 포지션 그룹별 Prospect 분석
position_group_analysis = train_fe.groupby('Position_Group').agg({
    'Prospect': ['count', 'sum', 'mean']
}).round(3)
position_group_analysis.columns = ['Count', 'Prospect_Count', 'Prospect_Rate']
position_group_analysis = position_group_analysis.sort_values('Prospect_Rate', ascending=False)
print('포지션 그룹별 Prospect 분석:')
position_group_analysis

### 16.4 핵심 특성공학 변수 분포 비교

In [ ]:
# 핵심 특성공학 변수들의 Prospect별 분포
key_fe_cols = ['Stats_Per_Age', 'Current_Position_Rating', 'Max_Field_Rating', 
               'Total_Stats_Sum', 'Technical_Skill', 'Mental_Skill',
               'Position_Fit_Score', 'Attack_Defense_Ratio', 'BMI']

fig, axes = plt.subplots(3, 3, figsize=(15, 12))

for idx, col in enumerate(key_fe_cols):
    ax = axes[idx // 3, idx % 3]
    train_fe[train_fe['Prospect'] == 0][col].hist(ax=ax, bins=30, alpha=0.5, label='Prospect=0', color='blue')
    train_fe[train_fe['Prospect'] == 1][col].hist(ax=ax, bins=30, alpha=0.5, label='Prospect=1', color='red')
    ax.set_title(f'{col} 분포')
    ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# 핵심 변수 Boxplot
fig, axes = plt.subplots(3, 3, figsize=(15, 12))

for idx, col in enumerate(key_fe_cols):
    ax = axes[idx // 3, idx % 3]
    train_fe.boxplot(column=col, by='Prospect', ax=ax)
    ax.set_title(f'{col} by Prospect')
    ax.set_xlabel('Prospect')

plt.suptitle('')
plt.tight_layout()
plt.show()

### 16.5 Work Rate 조합별 분석

In [ ]:
# Work Rate 조합별 Prospect 분석
workrate_analysis = train_fe.groupby('WorkRate_Combo').agg({
    'Prospect': ['count', 'sum', 'mean']
}).round(3)
workrate_analysis.columns = ['Count', 'Prospect_Count', 'Prospect_Rate']
workrate_analysis = workrate_analysis.sort_values('Prospect_Rate', ascending=False)
print('Work Rate 조합별 Prospect 분석:')
workrate_analysis

### 16.6 특성공학 요약

In [ ]:
print('='*70)
print('특성공학 요약')
print('='*70)

print(f'''
1. 생성된 변수 수: {len(new_cols)}개

2. 상관관계 상위 새로운 변수 (Top 10):''')
for i, (col, corr) in enumerate(new_corr.head(10).items(), 1):
    print(f'   {i:2d}. {col:30s}: {corr:+.4f}')

print(f'''
3. 추천 특성공학 변수:
   - Stats_Per_Age: 나이 대비 스탯 (유망주 핵심 지표)
   - Current_Position_Rating: 현재 포지션 레이팅
   - Max_Field_Rating: 최고 필드 레이팅
   - Total_Stats_Sum: 종합 스탯 합계
   - Technical_Skill, Mental_Skill: 기술/정신 능력 종합
   - Position_Fit_Score: 포지션 적합도
   - Position_Group: 포지션 대분류 (범주형)
   - WorkRate_Combo: Work Rate 조합 (범주형)

4. 새로운 범주형 변수:
   - Position_Group: {train_fe['Position_Group'].unique().tolist()}
   - Age_Group: {train_fe['Age_Group'].unique().tolist()}
   - WorkRate_Combo: {train_fe['WorkRate_Combo'].nunique()}개 조합

5. 모델링 시 주의사항:
   - 새로운 범주형 변수 인코딩 필요
   - 다중공선성 높은 변수 제거 고려 (Rating 관련)
   - train과 test에 동일한 특성공학 함수 적용 필수
''')

In [ ]:
# 특성공학 적용된 데이터 확인
train_fe.head()